In [ ]:
# @title Install Dependencies
# Install necessary libraries for Colab environment
!pip install yfinance pyts deap gym talib-binary

In [ ]:
# @title Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
import torch
import yfinance as yf
import talib
from pyts.image import GramianAngularField
from deap import base, creator, tools, gp, algorithms
import operator
import gym
from gym import spaces
import warnings

warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

In [ ]:
# @title Stage 1: Data Preparation
# 1. Get data (10 years minimum for robust patterns)
print("Loading data...")
ticker = "SPY"  # or your asset
df = yf.download(ticker, start="2014-01-01", end="2024-12-31", progress=False)

# Handle MultiIndex columns if present (yfinance update)
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

print(f"✓ Loaded {len(df)} days of data")

# 2. Compute technical indicators
# Ensure we have enough data
if len(df) > 50:
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    df['ATR'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['SMA20'] = talib.SMA(df['Close'], timeperiod=20)
    df['SMA50'] = talib.SMA(df['Close'], timeperiod=50)
    df['Volume_MA'] = df['Volume'].rolling(20).mean()
    
    # Drop NaNs created by indicators
    df.dropna(inplace=True)

# 3. Create GASF images (visual representation)
print("Creating GASF images...")
gasf = GramianAngularField(image_size=64, method='summation')

# Convert prices to returns (stationary)
returns = np.log(df['Close'] / df['Close'].shift(1)).dropna().values

# Create rolling 20-day windows as GASF images
window_size = 20
gasf_images = []
gasf_indices = []

# We need to align indices with the dataframe
# The returns array is 1 shorter than df (due to shift)
# We start loop from 0, which corresponds to df index 1
for i in range(len(returns) - window_size):
    window = returns[i:i+window_size].reshape(1, -1)
    try:
        gasf_img = gasf.fit_transform(window)
        gasf_images.append(gasf_img[0]) # fit_transform returns (n_samples, image_size, image_size)
        # The index in df corresponding to the END of this window
        # returns[0] is change from day 0 to 1.
        # window returns[i:i+20] covers days i+1 to i+20.
        # The prediction is for day i+21 onwards.
        gasf_indices.append(i + window_size) 
    except:
        pass

gasf_images = np.array(gasf_images)
print(f"✓ Created {len(gasf_images)} GASF images")

# 4. Create labels (future returns)
# We want to predict returns AFTER the window
df['Returns'] = np.log(df['Close'] / df['Close'].shift(1))
df['Future_5d_Return'] = df['Returns'].rolling(5).sum().shift(-5)
df['Label'] = (df['Future_5d_Return'] > 0).astype(int)

# Align data for subsequent stages
# gasf_indices points to the row in df that is the LAST day of the window.
# We want to predict Future_5d_Return from that day.
valid_indices = [idx for idx in gasf_indices if idx < len(df)]
aligned_future_returns = df['Future_5d_Return'].iloc[valid_indices].values
aligned_gasf_images = gasf_images[:len(valid_indices)]

# Remove NaNs from alignment (end of dataset)
mask = ~np.isnan(aligned_future_returns)
aligned_gasf_images = aligned_gasf_images[mask]
aligned_future_returns = aligned_future_returns[mask]
aligned_indices = np.array(valid_indices)[mask]

print(f"✓ Data ready for discovery. Samples: {len(aligned_gasf_images)}")

In [ ]:
# @title Stage 2: Unsupervised Visual Discovery
class VisualPatternFinder:
    """Cluster GASF images - find recurring visual structures."""
    
    def __init__(self, n_patterns=10):
        self.n_patterns = n_patterns
        self.kmeans = None
        self.patterns = {}
    
    def discover_visual_patterns(self, gasf_images, future_returns):
        """
        Cluster images → find which clusters are profitable.
        """
        # Reshape for clustering (flatten images)
        n_samples, h, w = gasf_images.shape
        X = gasf_images.reshape(n_samples, h*w)
        
        # Cluster
        self.kmeans = KMeans(n_clusters=self.n_patterns, random_state=42, n_init=10)
        clusters = self.kmeans.fit_predict(X)
        
        # Analyze each cluster
        print("\nVISUAL PATTERN ANALYSIS")
        print("=" * 60)
        
        for cluster_id in range(self.n_patterns):
            mask = clusters == cluster_id
            cluster_returns = future_returns[mask]
            
            if len(cluster_returns) == 0:
                continue

            # Statistics
            avg_return = np.mean(cluster_returns)
            win_rate = (cluster_returns > 0).mean()
            frequency = mask.sum() / len(clusters)
            std_dev = np.std(cluster_returns) + 1e-8
            sharpe = avg_return / std_dev * np.sqrt(252/5) # Annualized roughly
            
            print(f"\nPattern {cluster_id}:")
            print(f"  Frequency: {frequency*100:.1f}% of days")
            print(f"  Next-5d return: {avg_return*100:.2f}%")
            print(f"  Win rate: {win_rate*100:.1f}%")
            print(f"  Sharpe: {sharpe:.2f}")
            
            if avg_return > 0.005: # Threshold for "profitable"
                print(f"  ✓ PROFITABLE PATTERN FOUND")
            
            self.patterns[cluster_id] = {
                'avg_return': avg_return,
                'win_rate': win_rate,
                'frequency': frequency,
                'sharpe': sharpe,
                'centroid': self.kmeans.cluster_centers_[cluster_id].reshape(h, w)
            }
            
    def plot_patterns(self):
        """Visualize the centroids of discovered patterns."""
        if not self.patterns:
            print("No patterns discovered yet.")
            return
            
        fig, axes = plt.subplots(2, (self.n_patterns + 1) // 2, figsize=(15, 6))
        axes = axes.flatten()
        
        for i, (cluster_id, stats) in enumerate(self.patterns.items()):
            if i < len(axes):
                axes[i].imshow(stats['centroid'], cmap='rainbow', origin='lower')
                axes[i].set_title(f"P{cluster_id}: WR {stats['win_rate']:.2f}")
                axes[i].axis('off')
        plt.tight_layout()
        plt.show()

# Discover patterns
finder = VisualPatternFinder(n_patterns=8)
finder.discover_visual_patterns(aligned_gasf_images, aligned_future_returns)
finder.plot_patterns()

print("\n✓ DISCOVERED: Which price shapes predict returns")

In [ ]:
# @title Stage 3: Rare Structure Detection
class RareStateDetector:
    """Find anomalous market structures."""
    
    def __init__(self):
        self.pca = PCA(n_components=0.95) # Keep 95% variance
        self.iso_forest = IsolationForest(contamination=0.05, random_state=42)
    
    def find_rare_states(self, df):
        """Analyze: What rare states precede big moves?"""
        
        # Features
        feature_cols = ['RSI', 'ATR', 'Volume_MA', 'SMA20', 'SMA50']
        data_subset = df[feature_cols].copy()
        data_subset.dropna(inplace=True)
        
        features = data_subset.values
        
        # Standardize
        features = StandardScaler().fit_transform(features)
        
        # Reduce dimensions
        features_reduced = self.pca.fit_transform(features)
        
        # Find anomalies
        anomalies = self.iso_forest.fit_predict(features_reduced)
        # -1 is anomaly, 1 is normal
        anomaly_indices = np.where(anomalies == -1)[0]
        
        print("\nRARE STATE ANALYSIS")
        print("=" * 60)
        print(f"Found {len(anomaly_indices)} rare states (top 5%)")
        
        # For each rare state, check next returns
        # We need to map these indices back to the dataframe to get future returns
        # data_subset indices match df indices if we are careful
        
        rare_returns = []
        
        for idx in anomaly_indices:
            # Get the actual index in the dataframe
            df_idx = data_subset.index[idx]
            
            # Look up future return
            if df_idx in df.index:
                ret = df.loc[df_idx, 'Future_5d_Return']
                if not np.isnan(ret):
                    rare_returns.append(ret)
        
        rare_returns = np.array(rare_returns)
            
        if len(rare_returns) > 0:
            print(f"After rare states:")
            print(f"  Average return: {np.mean(rare_returns)*100:.2f}%")
            print(f"  Win rate: {(rare_returns > 0).mean()*100:.1f}%")
            print(f"  Max drawdown: {np.min(rare_returns)*100:.2f}%")
            
            if np.mean(rare_returns) < -0.01:
                print(f"  ⚠️ RARE STATES PRECEDE CRASHES!")
            elif np.mean(rare_returns) > 0.01:
                print(f"  ✓ RARE STATES PRECEDE RALLIES!")
            else:
                print(f"  ℹ️ Rare states are neutral/noisy.")

detector = RareStateDetector()
detector.find_rare_states(df)

In [ ]:
# @title Stage 4: Symbolic Regression (Genetic Programming)
class FormulEvolver:
    """Genetic programming: evolve trading formulas."""
    
    def __init__(self):
        # Check if creator classes already exist to avoid errors on re-run
        if not hasattr(creator, "FitnessMax"):
            creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        if not hasattr(creator, "Individual"):
            creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)
        
        self.toolbox = base.Toolbox()
        self.pset = gp.PrimitiveSet("MAIN", 5)  # 5 inputs: RSI, ATR, SMA20, SMA50, VolMA
        
        # Rename arguments for clarity in output
        self.pset.renameArguments(ARG0='RSI', ARG1='ATR', ARG2='SMA20', ARG3='SMA50', ARG4='VolMA')
        
        # Operations
        self.pset.addPrimitive(operator.add, 2, name='add')
        self.pset.addPrimitive(operator.sub, 2, name='sub')
        self.pset.addPrimitive(operator.mul, 2, name='mul')
        def protected_div(x, y):
            return x / (y + 1e-8)
        self.pset.addPrimitive(protected_div, 2, name='div')
        def protected_sqrt(x):
            return np.sqrt(np.abs(x))
        self.pset.addPrimitive(protected_sqrt, 1, name='sqrt')
        self.pset.addPrimitive(np.sin, 1, name='sin')
        self.pset.addPrimitive(np.tanh, 1, name='tanh')
        
        self.pset.addTerminal(1.0, name='const_1')
        self.pset.addTerminal(0.5, name='const_0.5')
        
        self.toolbox.register("expr", gp.genHalfAndHalf, pset=self.pset, min_=1, max_=3)
        self.toolbox.register("individual", tools.initIterate, creator.Individual, self.toolbox.expr)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        
        self.toolbox.register("evaluate", self.eval_formula)
        self.toolbox.register("mate", gp.cxOnePoint)
        self.toolbox.register("mutate", gp.mutUniform, expr=self.toolbox.expr, pset=self.pset)
        self.toolbox.register("select", tools.selTournament, tournsize=3)
        
        # Limit tree height to prevent bloat
        self.toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
        self.toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
    
    def eval_formula(self, individual, X, y):
        """Evaluate formula on data."""
        func = gp.compile(individual, self.pset)
        
        # X is (n_samples, 5)
        # We need to pass columns as arguments
        try:
            # Apply function to all rows
            # func expects 5 arguments. We can unpack columns.
            # Note: This might be slow if not vectorized. 
            # DEAP functions usually work on scalars, but numpy functions work on arrays.
            # If primitives are numpy-compatible, we can pass arrays directly.
            pred = func(X[:,0], X[:,1], X[:,2], X[:,3], X[:,4])
            
            # Check for NaNs or Infs
            if np.isnan(pred).any() or np.isinf(pred).any():
                return (-100.0,)
                
            # Metric: Correlation with future returns
            # We want high absolute correlation (can be negative, we just flip signal)
            # But here we optimize for positive correlation
            correlation = np.corrcoef(pred, y)[0, 1]
            
            if np.isnan(correlation):
                return (-100.0,)
            
            return (correlation,)
            
        except Exception as e:
            return (-100.0,)
    
    def evolve(self, X, y, pop_size=100, generations=10):
        """Evolve formulas."""
        pop = self.toolbox.population(n=pop_size)
        hof = tools.HallOfFame(5)
        
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", np.mean)
        stats.register("max", np.max)
        
        # Pass X and y to evaluate
        # We need to register evaluate with fixed arguments or use partial
        # But DEAP expects evaluate(individual). 
        # We can't easily pass X, y in register.
        # Hack: Bind X and y to the method before running
        self.toolbox.register("evaluate", self.eval_formula, X=X, y=y)
        
        pop, log = algorithms.eaSimple(
            pop, self.toolbox,
            cxpb=0.7, mutpb=0.2,
            ngen=generations,
            stats=stats,
            halloffame=hof,
            verbose=True
        )
        
        return hof, log

# Prepare data for Evolution
feature_cols = ['RSI', 'ATR', 'SMA20', 'SMA50', 'Volume_MA']
data_subset = df[feature_cols + ['Future_5d_Return']].dropna()

X_features = data_subset[feature_cols].values
y_returns = data_subset['Future_5d_Return'].values

# Normalize features for better evolution stability
X_features = StandardScaler().fit_transform(X_features)

# Evolve
print("\nFORMULA EVOLUTION")
print("=" * 60)

evolver = FormulEvolver()
# Reduced generations for demo speed
best_formulas, log = evolver.evolve(X_features, y_returns, pop_size=50, generations=10)

print("\nBest discovered formulas:")
for i, formula in enumerate(best_formulas[:3]):
    correlation = formula.fitness.values[0]
    print(f"\n{i+1}. Correlation: {correlation:.4f}")
    print(f"   Formula: {gp.stringify(formula)}")

In [ ]:
# @title Stage 5: RL with Curiosity
class TradingEnvWithCuriosity(gym.Env):
    """RL environment where agent learns to find profitable sequences."""
    
    def __init__(self, df, window=20):
        super(TradingEnvWithCuriosity, self).__init__()
        self.df = df.reset_index(drop=True)
        self.window = window
        self.current_idx = window
        self.max_idx = len(df) - 5 # Leave room for future return calc
        
        # Actions: 0=SELL, 1=HOLD, 2=BUY
        self.action_space = spaces.Discrete(3)
        
        # Observation: RSI, ATR, Vol, SMA diffs
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(5,), 
            dtype=np.float32
        )
    
    def reset(self):
        self.current_idx = np.random.randint(self.window, self.max_idx - 100)
        return self._get_state()
    
    def step(self, action):
        # Get returns for the NEXT day (or next 5 days)
        # Here we simulate a 1-day step
        current_return = self.df['Returns'].iloc[self.current_idx]
        
        # Reward logic
        # 0=SELL (short), 1=HOLD (flat), 2=BUY (long)
        # Map to -1, 0, 1
        position = action - 1 
        
        # Simple PnL reward
        reward = position * current_return * 100 
        
        # Curiosity/Entropy bonus could be added here (simplified for now)
        
        self.current_idx += 1
        done = self.current_idx >= self.max_idx
        
        return self._get_state(), reward, done, {}
    
    def _get_state(self):
        idx = self.current_idx
        # Ensure we don't go out of bounds
        if idx >= len(self.df):
            idx = len(self.df) - 1
            
        # Simple normalization on the fly
        rsi = self.df['RSI'].iloc[idx] / 100.0
        atr = self.df['ATR'].iloc[idx] / (self.df['Close'].iloc[idx] + 1e-8)
        vol = np.log(self.df['Volume'].iloc[idx] / (self.df['Volume_MA'].iloc[idx] + 1e-8) + 1e-8)
        sma20 = (self.df['Close'].iloc[idx] - self.df['SMA20'].iloc[idx]) / (self.df['SMA20'].iloc[idx] + 1e-8)
        sma50 = (self.df['Close'].iloc[idx] - self.df['SMA50'].iloc[idx]) / (self.df['SMA50'].iloc[idx] + 1e-8)
        
        return np.array([rsi, atr, vol, sma20, sma50], dtype=np.float32)

print("\nRL ENVIRONMENT SETUP")
print("=" * 60)
env = TradingEnvWithCuriosity(df)
obs = env.reset()
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
print("Initial observation:", obs)
print("Agent ready to explore.")

In [ ]:
# @title Stage 6: Multimodal Fusion
class EnsembleAlphaDiscovery:
    """Combine all discovery methods."""
    
    def __init__(self, visual_finder, rare_detector, best_formula):
        self.visual = visual_finder
        self.rare = rare_detector
        self.formula = best_formula
        self.pset = gp.PrimitiveSet("MAIN", 5) # Re-define or pass from evolver
        # Re-setup pset for compilation
        self.pset.renameArguments(ARG0='RSI', ARG1='ATR', ARG2='SMA20', ARG3='SMA50', ARG4='VolMA')
        self.pset.addPrimitive(operator.add, 2, name='add')
        self.pset.addPrimitive(operator.sub, 2, name='sub')
        self.pset.addPrimitive(operator.mul, 2, name='mul')
        def protected_div(x, y): return x / (y + 1e-8)
        self.pset.addPrimitive(protected_div, 2, name='div')
        def protected_sqrt(x): return np.sqrt(np.abs(x))
        self.pset.addPrimitive(protected_sqrt, 1, name='sqrt')
        self.pset.addPrimitive(np.sin, 1, name='sin')
        self.pset.addPrimitive(np.tanh, 1, name='tanh')
        self.pset.addTerminal(1.0, name='const_1')
        self.pset.addTerminal(0.5, name='const_0.5')
        
        self.compiled_formula = gp.compile(self.formula, self.pset)
    
    def get_ensemble_signal(self, current_state_dict, gasf_img):
        """Query all modalities."""
        
        votes = {}
        
        # 1. Visual Vote
        # Predict cluster
        if self.visual.kmeans:
            cluster = self.visual.kmeans.predict(gasf_img.reshape(1, -1))[0]
            pattern_stats = self.visual.patterns.get(cluster, {})
            # Vote is avg_return scaled
            votes['visual'] = np.sign(pattern_stats.get('avg_return', 0))
        else:
            votes['visual'] = 0
            
        # 2. Genetic Vote
        # Evaluate formula
        try:
            f_val = self.compiled_formula(
                current_state_dict['RSI'], 
                current_state_dict['ATR'], 
                current_state_dict['SMA20'], 
                current_state_dict['SMA50'], 
                current_state_dict['Volume_MA']
            )
            votes['genetic'] = np.sign(f_val)
        except:
            votes['genetic'] = 0
            
        # 3. Rare Vote
        # (Simplified: if rare, assume reversion or momentum based on history)
        # For now, random placeholder or neutral
        votes['rare'] = 0 
        
        consensus = np.mean(list(votes.values()))
        
        return {
            'signal': consensus,
            'votes': votes,
        }

print("✓ Ensemble ready: Uses all discovery methods")

In [ ]:
# @title Stage 7: Statistical Validation
def is_discovery_real(predictions, returns, n_permutations=100):
    """Is accuracy better than random?"""
    
    # Align lengths
    min_len = min(len(predictions), len(returns))
    predictions = predictions[:min_len]
    returns = returns[:min_len]
    
    # Calculate real accuracy (Directional)
    # predictions are -1, 0, 1. returns are float.
    # Match signs
    correct = np.sign(predictions) == np.sign(returns)
    real_accuracy = correct.mean()
    
    perm_accuracies = []
    for _ in range(n_permutations):
        shuffled = np.random.permutation(returns)
        acc = (np.sign(predictions) == np.sign(shuffled)).mean()
        perm_accuracies.append(acc)
    
    p_value = (np.array(perm_accuracies) >= real_accuracy).mean()
    
    print(f"Real accuracy: {real_accuracy:.4f}")
    print(f"Random accuracy: {np.mean(perm_accuracies):.4f}")
    print(f"P-value: {p_value:.4f}")
    
    return p_value < 0.05  # Significant?

print("Validation function ready.")